# Outlier Detection
Based on Charu Aggarwal, Data Mining, chapter 8, Outlier Analysis.

## Extreme Value Analysis
1. Not all outliers have extreme values. A point between distant clusters might be close to the global mean while being an outlier.
1. Easy to define "extreme" for univariate analysis. Use tails of the distribution(s).
1. Possible on multivariate but requires work. 
1. For single cluster, deal with different variance per axis, and covariance. Compute each point's Mahalanobis distance from the mean and variance. Equivalently, rotate the axis to remove covariance, and scale the data along each axis separately to normalize variance.
1. For multi-cluster data, compute each point's Mahalanobis distance to the mean and variance of its nearest cluster.

## Depth Analysis
1. Compute the convex hull of all points.
1. Iteratively, remove points on the perimeter, and compute the hull for the remainder.
1. This generates an onion-layer model of the data.
1. Score each point by the number of onion-layers from the center.
1. Fails with high dimensions. At extreme, every point is on the perimeter.

## Probability / GMM / EM
1. Build a probabilistic model, such as Gaussian Mixture (GMM).
1. Choose the number of distributions. 
1. Estimate means and variances, such as by Expectation Maximization (EM).
1. Compute the probability of each point. 
1. Define outliers as low-probability by some threshold.

EM is similar to K Means clustering. 
1. EM is "soft". Each point is accounted for by a portion of every distribution. Each point's probability is the sum of its probabilities under each distribution.
1. K Means is "hard". Each point is assigned entirely to its closest cluster.
1. In both methods, you iteratively assign points to clusters and re-estimate the cluster parameters.

## Clustering
1. Compute clusters by some algorithm, such as K Means.
1. Score each point by its distance to the nearest cluster mean.
1. In multivariate data, prefer Mahalanobis distance over Euclidean. 

Clustering uses global information (all the points). That makes it better than distance-based methods, which use local information (a few points). Example where it exceeds: The data has several large clusters, plus a small cluster of outliers. Distance methods don't see the outliers because they are all close to the other outliers. But good clustering would see all these points as far from any (big) cluster.

## Distance / kNN
Score each point by its distance from its neighbors. Label outliers by some threshold.

Two ways to score every point using k:
1. Distance to its kth nearest neighbor.
1. Average distance to its k nearest neighbors.

Distance methods are myopic, seeing local data only. Example where it fails: The data has sparse and dense clusters (of equal count). There is no single distance threshold that works for both types.

Distance methods require all-vs-all quadratic-time computation. However, apporoximation algorithms use tricks: sampling, pruning, early stopping. Also indexing (low dimensions only), but the book doesn't explain.

## Density / LOF / Histogram / Kernel Density
Define outliers as points in low-density regions. How to find those regions?

Simplest method uses histogram (univariate) or grid (multivariate). Bins with low counts are the outliers. Problems: sensitive to the bin size, discrete and very local, fails at higher dimensions.

Kernel density provides a continuous and global score. But it is a big compute. Assume a distribution such as Gaussian. The assumed variance is a smoothing parameter. For every point, consider that every other point could be the mean. The score for every point is the sum of its probabilties over every other point as the mean. If there is a true cluster, its points will contribute heavily to the score. 

LOF = Local Outlier Factor
1. Define reachability R(X,Y) = max (X-Y distance, X-K distance), where K is the kth nearest neighbor of X. This is a smoothing function that considers all "close enough" scores the same. This smoothing provides statistical stability.
1. Define average reachability AR(X) = mean R(X,Y) for all Y in the neighborhood. It seems to me this neighborhood should be larger than the k nearest neighbors. Otherwise, the mean is the max. 
1. Define LOF = mean{AR(X)/AR(Y)} for all Y in the neighborhood. If X is similarly dense to all its neighbors Y, the mean is 1. 
1. Define outlier as LOF>threshold. In practice, try several neighborhood sizes and use max LOF over these trials.
1. Aggarwal argues LOF should not be called a density measure because it never measures density proper. It is a relative distance measure.

## Information theory
1. Generate a code book for lossy compression. Compress the data, then regenerate the data. Outliers are those not predicted.
1. Compute entropy.
1. Compute Kolmogorov complexity.

## Evaluation by ROC
1. Score every point. Set a threshold for outliers. 
1. Obtain data labels from an oracle or some objective measure.
1. Compare predictions to labels to obtain TP and FP for each threshold value.
1. Use ROC to visualize the treshold effect. X-axis = FPR. Y-axis = TPR. The diagonal represents guessing. Curves closer to top left are better than guessing.
1. AUROC (area under the receiver operator curve) measures overall goodness.
1. Do not use these findings to adjust the outlier detection parameters. That encourages overfitting the few outliers in this sample. It inflates the accuracy compared to actual accuracy on unseen data.